# 3.6 Lab - Linear Regression

## 3.6.1 Importing Packages

We start by importing the needed standard libraries at the top level.

In [2]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from statsmodels.stats.outliers_influence \
    import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)

#### Inspecting Objects and Namespaces

The function `dir()` provides a list of objects in a namespace.

In [3]:
dir()

['In',
 'MS',
 'Out',
 'VIF',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'anova_lm',
 'exit',
 'get_ipython',
 'load_data',
 'np',
 'open',
 'pd',
 'poly',
 'quit',
 'sm',
 'subplots',
 'summarize']

This shows everything that `Python` can find at the top level. There are certain objects like `__builtins__` that contain references to built-in functions like `print()`.

Every Python object has its own notion of namespace, also accessible with `dir()`. This will include the attributes of the object as well as any methods associated with it. For example, we can see `sum` in the listing for an array.

In [4]:
A = np.array([3, 5, 11])
dir(A)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_namespace__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__buffer__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',


This indicates that the object `A.sum` exists. IT is a method that can be used to compute the sum of the array `A` as seen typing `A.sum?`.

In [5]:
A.sum?

Docstring:
a.sum(axis=None, dtype=None, out=None, keepdims=False, initial=0, where=True)

Return the sum of the array elements over the given axis.

Refer to `numpy.sum` for full documentation.

See Also
--------
numpy.sum : equivalent function
Type:      builtin_function_or_method

In [6]:
A.sum()

np.int64(19)

## 3.6.2 Simple Linear Regression

In this section we will construct **model matrices** (or **design matrices**) using the `ModelSpec()` transform from `ISLP.models`. We will use the `Boston` dataset (see 2.4 Exercises for full details on the Boston dataset).

The Python model `statsmodel` contains functions implementing several commonly used regression methods. The `load_data()` function in the `ISLP` package loads in the data for us automatically.

In [10]:
Boston = load_data("Boston")
Boston.columns

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'lstat', 'medv'],
      dtype='object')

We start by using the `sm.OLS()` function to fit a simple linear regression model. The response will be `medv` and `lstat` will be the single predictor. For this model, we can create the model matrix by hand.

In [13]:
# Establish the intercept value and extract the lstat predictor
X = pd.DataFrame({'intercept' : np.ones(Boston.shape[0]),
                  'lstat': Boston['lstat']})
X[:4]

,intercept,lstat
0,1.0,4.98
1,1.0,9.14
2,1.0,4.03
3,1.0,2.94


Next, we extract the response and fit the model.

In [14]:
y = Boston['medv']      # Extract the response
model = sm.OLS(y, X)    # Initiate the model
results = model.fit()   # Fit the model

The `ISLP` function `summarize()` produces a simple table of the parameter estimates, their standard errors, t-statistics, and p-values. It takes in a single argument, namely the `results` returned from the `fit` method above.

In [15]:
# Summarize the results
summarize(results)

,coef,std err,t,P>|t|
intercept,34.5538,0.563,61.415,0.0
lstat,-0.9500,0.039,-24.528,0.0


Next we outline a more useful and general framework for constructing a model matrix `X`.

#### Using Transformations: Fit and Transform